In [7]:
import os
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
#import dlib
import datetime as dt

dlib tidak dapat diinstal karena dependency yang dibutuhkan melebihi kapasitas PC saya, dan beberapa soalan juga tidak dapat dilaksanakan karena membutuhkan alokasi storage yang tinggi

![storage](https://cdn.discordapp.com/attachments/644704134078922771/1312076221428793424/Screenshot_258.png?ex=674b2db4&is=6749dc34&hm=6df98e56d86e19b88d95924c28c33d0b2354da9ceceabb7e1306b749dae081f1&)

In [4]:
# Paths
VIDEO_PATH = os.path.join(os.getcwd(), 'data', 'Fajrito.mp4')  # Path to the video file
FRAMES_DIR = os.path.join(os.getcwd(), 'data', 'fjr')      # Directory to save frames

# Create directory for frames if it doesn't exist
if not os.path.exists(FRAMES_DIR):
    os.makedirs(FRAMES_DIR)

# Open video file
cap = cv2.VideoCapture(VIDEO_PATH)

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:  # Break the loop if there are no more frames
        break
    # Save frame as an image
    frame_filename = os.path.join(FRAMES_DIR, f'frame_{frame_count:03d}.jpg')
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()
print(f"{frame_count} frames have been saved to {FRAMES_DIR}")

1914 frames have been saved to d:\mgodonf\if4021-handson-main\data\fjr


In [8]:
IMGS_PATH = (os.path.join(os.getcwd(), 'data', 'fjr'))
list_imgs = glob(IMGS_PATH + '/*.jpg')
print(f"5 path pertama: {list_imgs[:5]}")

5 path pertama: ['d:\\mgodonf\\if4021-handson-main\\data\\fjr\\frame_000.jpg', 'd:\\mgodonf\\if4021-handson-main\\data\\fjr\\frame_001.jpg', 'd:\\mgodonf\\if4021-handson-main\\data\\fjr\\frame_002.jpg', 'd:\\mgodonf\\if4021-handson-main\\data\\fjr\\frame_003.jpg', 'd:\\mgodonf\\if4021-handson-main\\data\\fjr\\frame_004.jpg']


In [9]:
list_imgs = sorted(list_imgs, key=lambda x: int(x.split('/')[-1].split('.')[0]))
print(f"5 path pertama setelah diurutkan: {list_imgs[:5]}")
print(f"Total jumlah gambar: {len(list_imgs)}")

ValueError: invalid literal for int() with base 10: 'd:\\mgodonf\\if4021-handson-main\\data\\fjr\\frame_000'

1. list_imgs adalah fungsi untuk mensortir seluruh frame yang telah dirender sebagai jpg.

key mengurutkan sortiran dari nama file.

x.split('/') Memisahkan path file berdasarkan karakter / untuk mendapatkan bagian terakhir dari path (nama file).

x.split('/')[-1].split('.')[0] Memisahkan nama file berdasarkan titik (.) untuk menghilangkan ekstensi.

In [10]:
images = []
for img_path in list_imgs:
    img = cv2.imread(img_path)
    images.append(img)

images_array = np.array(images)

print(f"Shape of images_array: {images_array.shape}")

MemoryError: Unable to allocate 11.1 GiB for an array with shape (1914, 1080, 1920, 3) and data type uint8

In [15]:
save_loc = os.path.join(os.getcwd(), 'data', 'Fajrito.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height, width, layers = images_array[0].shape
video = cv2.VideoWriter(save_loc, fourcc, 30, (width, height))

for image in images_array:
    video.write(image)

video.release()

IndexError: index 0 is out of bounds for axis 0 with size 0

2. Fourcc (Four Character Code) adalah fungsi untuk mengspesifikasikan codec yang digunakan untuk merender video, yang digunakan disini adalah mp4v (MPEG-4 codec).

codec lain dapat digunakan di dalam cv2.VideoWriter_fourcc untuk codec lain, contoh berupa XVID (.avi), H264 (.mp4), dan MJPG (.jpeg)

In [ ]:
VID_PATH = os.path.join(os.getcwd(), 'data', 'Fajrito.mp4')
IMGSEQ_PATH = os.path.join(os.getcwd(), 'data', 'fjr-imgseq')

if not os.path.exists(IMGSEQ_PATH):
    os.makedirs(IMGSEQ_PATH)

cap = cv2.VideoCapture(VID_PATH)
frame_count = 0

while frame_count < 100:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(os.path.join(IMGSEQ_PATH, f'frame_{frame_count:03d}.jpg'), frame)
    frame_count += 1

cap.release()
print("100 frame pertama berhasil disimpan.")

Lower fps

In [14]:
VID_PATH = os.path.join(os.getcwd(), 'data', 'Fajrito.mp4')
IMGSEQ_PATH = os.path.join(os.getcwd(), 'data', 'fjr-imgseq')
SAVE_VIDEO_PATH = os.path.join(os.getcwd(), 'data', 'video_low_fps.mp4')

# Create directory for frames
if not os.path.exists(IMGSEQ_PATH):
    os.makedirs(IMGSEQ_PATH)



# Capture video
cap = cv2.VideoCapture(VID_PATH)
original_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_skip = 3
total_frames = original_frames // frame_skip
frame_count = 0
saved_count = 0

frames_to_save = []

# Process every 3rd frame
while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_count % 3 == 0:  # Save every 3rd frame
        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Resize to 1280x720
        gray_frame = cv2.resize(gray_frame, (1280, 720))
        
        # Convert back to 3-channel RGB
        rgb_frame = np.stack([gray_frame] * 3, axis=-1)
        
        # Add a moving red dot
        dot_position = int((saved_count / (total_frames - 1)) * (1280 - 1))  # Horizontal position of dot
        dot_position = min(dot_position, 1279)
        
        rgb_frame[360, dot_position, :] = [255, 0, 0]
        
        # Save frame to folder
        frame_path = os.path.join(IMGSEQ_PATH, f'frame_{saved_count:03d}.jpg')
        cv2.imwrite(frame_path, rgb_frame)
        frames_to_save.append(rgb_frame)
        saved_count += 1
    frame_count += 1

cap.release()
print(f"{saved_count} frames berhasil disimpan.")

# Create video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height, width, _ = frames_to_save[0].shape
video = cv2.VideoWriter(SAVE_VIDEO_PATH, fourcc, 10, (width, height))  # New FPS = 10

for frame in frames_to_save:
    video.write(frame)

video.release()
print(f"Video berhasil disimpan: {SAVE_VIDEO_PATH}")

638 frames berhasil disimpan.
Video berhasil disimpan: d:\mgodonf\if4021-handson-main\data\video_low_fps.mp4
